# |PIX Forecasting - Modeling| Performing Out-of-sample inference.

**Objective**: Load the production logged models and predict the next forecast horizon periods.


## 1.0 Imports

### 1.1 Setting working directory

In [1]:
import sys
import os

if not os.getcwd().split("\\")[-1] == "ip_forecasting":
    notebook_dir = os.path.dirname(
        os.path.abspath("__file__")
    )
    project_root = os.path.abspath(os.path.join(notebook_dir, "../"))
    os.chdir(project_root)

In [3]:
import pandas as pd
import numpy as np
import pandas_gbq
import locale

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mlflow
import mlflow.sklearn

import src.utils.useful_functions as uf
from src.data.data_loader import load_and_preprocess_model_dataset
from src.models.evaluate import *
from src.visualization.data_viz import *
from scripts.run_inference import *
from src.features.feat_eng import *
from src.visualization.data_viz import plot_multiple_models_forecast

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

### 1.2 Parameter setting

In [3]:
TARGET_COL          = model_config["target_col"]
PREDICTED_COL       = model_config["predicted_col"]
FORECAST_HORIZON    = model_config["forecast_horizon"]
N_SPLITS            = model_config["n_windows"]
MODEL_NAME          = model_config["model_name"]
USE_TUNED_PARMS     = model_config["use_tuned_params"]
EXPERIMENT_PATH     = model_config["mlflow_experiment_path_production"]
models_list         = list(get_models().keys())

## 2.0 Data Loading

In [4]:
feature_df = load_and_preprocess_model_dataset("featurized_df")
feature_df = feature_df.set_index("date")

## 3.0 Out of Sample Forecasting

In [5]:
final_predictions_df, future_df, shap_values_inference = inference_pipeline(
    processed_ip_dataset    = feature_df,
    models_list             = models_list,
    model_name              = MODEL_NAME,
    experiment_path         = EXPERIMENT_PATH,
    write_to_table          = True,
    log_to_experiment       = True,
)

100%|██████████| 1/1 [00:00<00:00, 133.05it/s]


Function 'inference_pipeline' executed in 127.14 seconds.


In [6]:
matplotlib.rcParams.update({"font.size": 13})

In [7]:
final_predictions_df.sample(5)

,date,index_price_lag_1,index_price_lag_2,index_price_sma_3,index_price_sma_12,comparison_index_price_shift_4_5,bband_spread_8,bband_spread_20,index_vs_market_price_lag_6,imports_lag_6,inventories_lag_16,inventories_lag_20,europulp_lag_6,europulp_lag_14,second_market_price_lag_3,final_product_price_lag_2,FORECAST,WEEK,PIX_REFERENECE,MODEL_TYPE,MODEL_FLAVOR,RUN_DATE,RUN_DATETIME,MODEL_URI
5,2025-01-10,558.181478,558.612797,558.045922,555.374040,0.996601,19.584072,59.038493,17.29,1.526127e+09,2011.28950,2088.1364,1501.29500,1571.25000,519.778102,6006.560918,555.825617,6,545.88,LGBMRegressor,mix,2025-01-27,2025-01-27 13:40:26.070553,file:///C:/Users/miche/Documents/projects/ip_f...
0,2024-12-06,545.880000,545.880000,548.853333,558.870833,99.800356,24.680949,232.514607,2.87,1.526127e+09,2063.23160,1961.6780,1518.78375,1454.66460,536.150000,6150.000000,545.335500,1,545.88,RandomForestRegressor,mix,2025-01-27,2025-01-27 13:39:48.650935,file:///C:/Users/miche/Documents/projects/ip_f...
2,2024-12-20,554.998781,556.891932,552.590238,556.153393,0.983922,18.603126,114.266015,11.32,1.526127e+09,2113.04120,2013.4220,1501.29500,1516.22325,528.590000,6064.165944,557.343490,3,545.88,LGBMRegressor,mix,2025-01-27,2025-01-27 13:40:26.070553,file:///C:/Users/miche/Documents/projects/ip_f...
2,2024-12-20,546.166992,548.120544,546.722512,554.686461,0.983922,20.293239,116.999241,11.32,1.526127e+09,2113.04120,2013.4220,1501.29500,1516.22325,528.590000,6064.165944,547.042969,3,545.88,XGBRegressor,mix,2025-01-27,2025-01-27 13:40:06.286449,file:///C:/Users/miche/Documents/projects/ip_f...
4,2025-01-03,548.093201,547.042969,547.101074,552.401976,1.004104,14.210801,83.155693,11.18,1.526127e+09,2074.61775,2063.2316,1501.29500,1552.90775,521.882436,6017.137556,547.616455,5,545.88,XGBRegressor,mix,2025-01-27,2025-01-27 13:40:06.286449,file:///C:/Users/miche/Documents/projects/ip_f...


In [44]:
fig = plot_multiple_models_forecast(
    training_dataframe=feature_df,
    forecast_dataframe=final_predictions_df,
    x_col="date",
    y_col=[TARGET_COL, PREDICTED_COL],
    title=f"Multiple models Forecast result.\n Inference date: {final_predictions_df['RUN_DATE'].unique()[0]}",
    x_label="Date",
    y_label="Target and Predicted Values"
)
fig.show()